In [9]:
import pandas as pd
from scipy.stats import binom_test, fisher_exact
from genepy.utils import helper as h 
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


## Processing the data

In [4]:
project='FishSuperResColoc'
version='v2'

In [11]:
rename = {
"Text Between Delimiters":"subset",
"Folder Short":"folder_short",
"Folder Path":"folder",
"Source.Name":"name",
"ImageDocumentName::Image Name":"image",
"ParentID::ID of the parent!!I":"parent_id",
"ID::ID!!I":"id",
"ImageSceneName::Image Scene Name ":"scene",
"Area1::Area Unscaled!!R":"area_u",
"Area::Area!!R": "area",
"CenterX1::Center X Unscaled!!R":"x",
"CenterY1::Center Y Unscaled!!R":"y",
"ImageIndexZ::Image Index Z!!I":"z",
'Classes 5 RegionsCount ::Classes 5 Count!!I':"count_red",
'Classes 9 RegionsCount ::Classes 9 Count!!I':"count_green",
"IntensityMaximum_TV1-T1-SR::Intensity Maximum of channel 'TV1-T1-SR'!!R":"max_red",
"IntensityMaximum_TV2-T1-SR::Intensity Maximum of channel 'TV2-T1-SR'!!R":"max_tv2_t1",
"IntensityMaximum_TV2-T2-SR::Intensity Maximum of channel 'TV2-T2-SR'!!R":"max_green",
"IntensityMaximum_TV2-T3-SR::Intensity Maximum of channel 'TV2-T3-SR'!!R":"max_dapi",
"IntensityMean_TV1-T1-SR::Intensity Mean Value of channel 'TV1-T1-SR'!!R":"mean_red",
"IntensityMean_TV2-T1-SR::Intensity Mean Value of channel 'TV2-T1-SR'!!R":"mean_tv2_t1",
"IntensityMean_TV2-T2-SR::Intensity Mean Value of channel 'TV2-T2-SR'!!R":"mean_green",
"IntensityMean_TV2-T3-SR::Intensity Mean Value of channel 'TV2-T3-SR'!!R":"mean_dapi",
"IntensityMinimum_TV1-T1-SR::Intensity Minimum of channel 'TV1-T1-SR'!!R":"min_red",
"IntensityMinimum_TV2-T1-SR::Intensity Minimum of channel 'TV2-T1-SR'!!R":"min_tv2_t1",
"IntensityMinimum_TV2-T2-SR::Intensity Minimum of channel 'TV2-T2-SR'!!R":"min_green",
"IntensityMinimum_TV2-T3-SR::Intensity Minimum of channel 'TV2-T3-SR'!!R":"min_dapi",
"IntensitySum0_TV1-T1-SR::Intensity Pixel Count of channel 'TV1-T1-SR'!!R":"pixsum_red",
"IntensitySum0_TV2-T1-SR::Intensity Pixel Count of channel 'TV2-T1-SR'!!R":"pixsum_tv2_t1",
"IntensitySum0_TV2-T2-SR::Intensity Pixel Count of channel 'TV2-T2-SR'!!R":"pixsum_green",
"IntensitySum0_TV2-T3-SR::Intensity Pixel Count of channel 'TV2-T3-SR'!!R":"pixsum_dapi",
"IntensityRange_TV1-T1-SR::Intensity Range of channel 'TV1-T1-SR'!!R":"range_red",
"IntensityRange_TV2-T1-SR::Intensity Range of channel 'TV2-T1-SR'!!R":"range_tv2_t1",
"IntensityRange_TV2-T2-SR::Intensity Range of channel 'TV2-T2-SR'!!R":"range_green",
"IntensityRange_TV2-T3-SR::Intensity Range of channel 'TV2-T3-SR'!!R":"range_dapi",
"IntensityStd_TV1-T1-SR::Intensity Standard Deviation of channel 'TV1-T1-SR'!!R":"std_red",
"IntensityStd_TV2-T1-SR::Intensity Standard Deviation of channel 'TV2-T1-SR'!!R":"std_tv2_t1",
"IntensityStd_TV2-T2-SR::Intensity Standard Deviation of channel 'TV2-T2-SR'!!R":"std_green",
"IntensityStd_TV2-T3-SR::Intensity Standard Deviation of channel 'TV2-T3-SR'!!R":"std_dapi",
"IntensitySum1_TV1-T1-SR::Intensity Sum of channel 'TV1-T1-SR'!!R":"sum_red",
"IntensitySum1_TV2-T1-SR::Intensity Sum of channel 'TV2-T1-SR'!!R":"sum_tv2_t1",
"IntensitySum1_TV2-T2-SR::Intensity Sum of channel 'TV2-T2-SR'!!R":"sum_green",
"IntensitySum1_TV2-T3-SR::Intensity Sum of channel 'TV2-T3-SR'!!R":"sum_dapi",
"IntensitySum2_TV1-T1-SR::Intensity Sum Squares of channel 'TV1-T1-SR'!!R":"sum2_red",
"IntensitySum2_TV2-T1-SR::Intensity Sum Squares of channel 'TV2-T1-SR'!!R":"sum2_tv2_t1",
"IntensitySum2_TV2-T2-SR::Intensity Sum Squares of channel 'TV2-T2-SR'!!R":"sum2_green",
"IntensitySum2_TV2-T3-SR::Intensity Sum Squares of channel 'TV2-T3-SR'!!R":"sum2_dapi",
"Unnamed: 27":"unknown"
}

In [80]:
data = pd.read_csv('../data/'+project+'/data_query_files/querry.csv').rename(columns=rename)
data

,subset,folder_short,folder,name,image,parent_id,id,scene,area,x,...,sum_1,sum_2,range_1,range_2,std_1,std_2,sum1_1,sum1_2,sum2_1,sum2_2
0,Subset10,AAVS1_CRISPR_MED1\AAVS1_CRISPR_MED1 Image7\,C:\Users\M232498\Desktop\Ju\GDrive\IF FISH fin...,Image 7_Subset10_Out_Maximum intensity project...,NaN,NaN,NaN,NaN,pixel²,pixel,...,pixel²,pixel²,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown²,Unknown²
1,Subset10,AAVS1_CRISPR_MED1\AAVS1_CRISPR_MED1 Image7\,C:\Users\M232498\Desktop\Ju\GDrive\IF FISH fin...,Image 7_Subset10_Out_Maximum intensity project...,Image 7_Subset10_Out_Maximum intensity project...,14.0,15.0,NaN,7,368.64285714286,...,7,7,1661,4080,547.72403548468,1510.241245314,49094,164490,346117272,3878964986
2,Subset10,AAVS1_CRISPR_MED1\AAVS1_CRISPR_MED1 Image7\,C:\Users\M232498\Desktop\Ju\GDrive\IF FISH fin...,Image 7_Subset10_Out_Maximum intensity project...,Image 7_Subset10_Out_Maximum intensity project...,14.0,16.0,NaN,28,396.21428571429,...,28,28,8089,13818,2760.2006509669,3701.1435641623,123989,523204,754750539,10146373720
3,Subset10,AAVS1_CRISPR_MED1\AAVS1_CRISPR_MED1 Image7\,C:\Users\M232498\Desktop\Ju\GDrive\IF FISH fin...,Image 7_Subset10_Out_Maximum intensity project...,Image 7_Subset10_Out_Maximum intensity project...,14.0,17.0,NaN,13,426.88461538462,...,13,13,4583,6576,1313.3590522016,2101.7813019022,108303,285907,922971237,6340918481
4,Subset10,AAVS1_CRISPR_MED1\AAVS1_CRISPR_MED1 Image7\,C:\Users\M232498\Desktop\Ju\GDrive\IF FISH fin...,Image 7_Subset10_Out_Maximum intensity project...,Image 7_Subset10_Out_Maximum intensity project...,14.0,18.0,NaN,29,340.74137931035,...,29,29,3515,12083,1077.0374837392,3107.8328355138,196046,460225,1357791794,7574132899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117757,Subset9,IF FISH final\Runx2_excel\,C:\Users\M232498\Desktop\Ju\GDrive\IF FISH fin...,Runx2_488_MYC_FISH_SIM-Orthogonal Projection-0...,Runx2_488_MYC_FISH_SIM-Orthogonal Projection-0...,1.0,23.0,NaN,55230,1146.4196089082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117758,Subset9,IF FISH final\Runx2_excel\,C:\Users\M232498\Desktop\Ju\GDrive\IF FISH fin...,Runx2_488_MYC_FISH_SIM-Orthogonal Projection-0...,Runx2_488_MYC_FISH_SIM-Orthogonal Projection-0...,1.0,24.0,NaN,60384,1546.5754835718,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117759,Subset9,IF FISH final\Runx2_excel\,C:\Users\M232498\Desktop\Ju\GDrive\IF FISH fin...,Runx2_488_MYC_FISH_SIM-Orthogonal Projection-0...,Runx2_488_MYC_FISH_SIM-Orthogonal Projection-0...,1.0,25.0,NaN,48042,2480.9880521211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117760,Subset9,IF FISH final\Runx2_excel\,C:\Users\M232498\Desktop\Ju\GDrive\IF FISH fin...,Runx2_488_MYC_FISH_SIM-Orthogonal Projection-0...,Runx2_488_MYC_FISH_SIM-Orthogonal Projection-0...,1.0,26.0,NaN,70306,1932.6927004808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
data.columns

Index(['subset', 'folder_short', 'folder', 'name', 'image', 'parent_id', 'id',
       'scene', 'area', 'x', 'y', 'max_1', 'max_2', 'mean_1', 'mean_2',
       'min_1', 'min_2', 'sum_1', 'sum_2', 'range_1', 'range_2', 'std_1',
       'std_2', 'sum1_1', 'sum1_2', 'sum2_1', 'sum2_2'],
      dtype='object')

In [5]:
cat ../data/$project/data_query_files/querry.csv | less

<U+FEFF>Text Between Delimiters,Folder Short,Folder Path,Source.Name,ImageDocumentName::Image Name,ParentID::ID of the parent!!I,ID::ID!!I,ImageSceneName::Image Scene Name ,Area1::Area Unscaled!!R,CenterX1::Center X Unscaled!!R,CenterY1::Center Y Unscaled!!R,IntensityMaximum_TV1-T1-SR::Intensity Maximum of channel 'TV1-T1-SR'!!R,IntensityMaximum_TV2-T1-SR::Intensity Maximum of channel 'TV2-T1-SR'!!R,IntensityMean_TV1-T1-SR::Intensity Mean Value of channel 'TV1-T1-SR'!!R,IntensityMean_TV2-T1-SR::Intensity Mean Value of channel 'TV2-T1-SR'!!R,IntensityMinimum_TV1-T1-SR::Intensity Minimum of channel 'TV1-T1-SR'!!R,IntensityMinimum_TV2-T1-SR::Intensity Minimum of channel 'TV2-T1-SR'!!R,IntensitySum0_TV1-T1-SR::Intensity Pixel Count of channel 'TV1-T1-SR'!!R,IntensitySum0_TV2-T1-SR::Intensity Pixel Count of channel 'TV2-T1-SR'!!R,IntensityRange_TV1-T1-SR::Intensity Range of channel 'TV1-T1-SR'!!R,IntensityRange_TV2-T1-SR::Intensity Range of channel 'TV2-T1-SR'!!R,IntensityStd_TV1-T1-SR::Int

In [ ]:
data.iloc[0].tolist()

In [ ]:
set(data.folder_short)

In [ ]:
cond = {'AAVS1_CRISPR_MED1\\AAVS1_CRISPR_MED1 Image7\\':'AAVS1',
 'IF FISH final\\AAVS1_CRISPR_MEF2D\\':'AAVS1',
 'IF FISH final\\IRF2BP2_excel\\':'',
 'IF FISH final\\MEF2C_excel\\':'',
 'IF FISH final\\MEF2D_CRISPR_MED1\\':'MEF2D',
 'IF FISH final\\MEF2D_CRISPR_MEF2D\\':'MEF2D',
 'IF FISH final\\Runx2_excel\\':''}

In [ ]:
prot = {'AAVS1_CRISPR_MED1\\AAVS1_CRISPR_MED1 Image7\\':'MED1',
 'IF FISH final\\AAVS1_CRISPR_MEF2D\\':'MEF2D',
 'IF FISH final\\IRF2BP2_excel\\':'IRF2BP2',
 'IF FISH final\\MEF2C_excel\\':'MEF2C',
 'IF FISH final\\MEF2D_CRISPR_MED1\\':'MED1',
 'IF FISH final\\MEF2D_CRISPR_MEF2D\\':'MEF2D',
 'IF FISH final\\Runx2_excel\\':'RUNX2'}

In [ ]:
data = data[~data.name.str.contains('Classes')]

In [ ]:
data['condition'] = [cond[i] for i in data.folder_short]

In [ ]:
data['protein'] = [prot[i] for i in data.folder_short]

In [ ]:
data['type'] = [i.split('.')[0].split('_')[-1] for i in data.name]

In [ ]:
data['name'] = [cond[i]+'_'+prot[i] for i in data.folder_short]

In [ ]:
set(data.type)

In [ ]:
data = data[~data.id.isna()]

In [ ]:
set(data.subset)

In [ ]:
data = data.drop(columns=['scene','image','folder','folder_short'])

In [ ]:
data['subset'] = [i.split('Subset')[-1] for i in data.subset]

In [ ]:
for val in ['area','x','y','max_1','max_2','mean_1','mean_2','min_1','min_2','range_1','range_2','std_1','std_2','sum_1','sum_2','sum1_1','sum1_2','sum2_1','sum2_2']:
    data[val]= data[val].astype(float)

In [ ]:
for val in ['subset','parent_id','id']:
    data[val] = data[val].astype('int')

In [ ]:
len(data.id)

# results

## why we are not doing a monte carlo but just a hypthesis testing on a binomial

monte carlo would be useful in our case if we did not know the expected distribution.
given a way to generate our distribution (which specific parameters can be learnt from the data) we might be able to express what would be the null hypothesis by estimating this unknown distribution. it is useful for when we don't know the exact distribution or when it is too complex to analytically solve.  

However in our case we can approximate the distribution well enough by a binomial. It is true that given that the dna will not have the same density everywhere on the nucleus, that the dots will have themselves variable intensity and size, which might express or not a multiplicity of points, the distribution is not a binomial. 

But you said yourself that for all purposes we could simplify these hypothesis. 

I also don't think that we have enough data points to correctly estimate the monte carlo parameters. 

A binomial is thus enough for this analysis

### having specificities in the data needing to be modelled:

I found that the data has specific bias that still need to be taken in account while not needing to change the statistics:

the avg size of a MYC dot is 350 whereas the one of a MYC green is 45

In [ ]:
def desc(data):
    return data.sum()/data.mean(),len(data)
# for each group
res = {}
for val in set(data.name):
    print('\n',val)
    group = data[data.name==val]
    space = group[group.type=='Nucleus'].area.sum()
    totsize = group[group.type==' Green Dots'].area.sum()
                 
    coloc,s_coloc = desc(group[group.type=='MYC  Green'].area)
    outside,s_outside = desc(group[group.type=='MYC'].area)
    print('data: \n  - total nucleuses size: '+str(int(space))+'\n  - total Green dot size: '+str(int(totsize))+'\n  - counts for coloc: '+str(int(s_coloc))+'\n  - counts for not coloc: '+str(int(s_outside))) 
    p_in = totsize/space
    isin = coloc/(outside+coloc)
    res[val] = [s_coloc,s_outside]
    print('proba: (null, obs)',p_in,isin)
    print('nb of datapoints: ',s_coloc+s_outside)
    print('p_value: ',binom_test([s_coloc,s_outside],p=p_in))

from this analysis it seems clear that the colocalization happens less than 50% of the time in any condition but that we have colocalization with MYC for all analyzed proteins compared to random chance, given the data extracted by the algorithm and the assumption we made.

In [ ]:
fisher_exact([res['AAVS1_MED1'], res['MEF2D_MED1']])

In [ ]:
fisher_exact([res['AAVS1_MEF2D'], res['MEF2D_MEF2D']])

we thus have much 1.7 times more colocalization of MEF2D when MEF2D is degraded vs when it is not and 2 times less colocalization of MED1 but p_val of .1

## running new version

In [ ]:
TODO: compare intensities & areas between conditions

In [12]:
project='FishSuperResColoc'
version='v2'

In [14]:
csvs = ! ls ../data/$project/yaser_analysis/*/*/*Region.csv
res = pd.DataFrame()
for val in csvs:
    v = pd.read_csv(val,).drop(index=0)
    v['filename'] = '-'.join(val.split('/')[-3].split('-')[1:])+"-w"+val.split('/')[-2].split('-')[-1]
    res = res.append(v)
xmls = ! ls ../data/$project/yaser_analysis/*/*/*Region.xml
for val in xmls:
    v = h.readXMLs(file = val)
    v['filename'] = '-'.join(val.split('/')[-3].split('-')[1:])+"-w"+val.split('/')[-2].split('-')[-1]
    res = res.append(v)
res = res.rename(columns=rename)

reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file
reading 1 file


In [15]:
res.parent_id = res.parent_id.replace(' ', '-1').astype(int)
res.id = res.id.replace(' ', '-1').astype(int)
res.area = res.area.astype(float)
res = res.reset_index(drop=True)

In [16]:
set(['-'.join(i.split('/')[-3].split('-')[1:])+"-w"+i.split('/')[-2].split('-')[-1] for i in csvs] + ['-'.join(i.split('/')[-3].split('-')[1:])+"-w"+i.split('/')[-2].split('-')[-1] for i in xmls])

{'VHL_MED1-MYC-wDMSO',
 'VHL_MED1-MYC-wVHL',
 'VHL_MEF2C-MEF2D-wDMSO',
 'VHL_MEF2C-MEF2D-wVHL',
 'VHL_MEF2C-MYC-wDMSO',
 'VHL_MEF2C-MYC-wVHL',
 'VHL_MEF2D-MYC-wDMSO',
 'VHL_MEF2D-MYC-wVHL'}

MEF2C red / MEF2D green for MEF2C-MEF2D experiment (no FISH!) else MYC red

TV2-T3 nucleus: Dapi

TV1-T1 red
TV2-T2 green

In [17]:
values =  ['area', 'x', 'y', "z", "count_red", "count_green", "max_red", "max_green", "max_dapi", "mean_red", "mean_green", "mean_dapi", "min_red", "min_green", "min_dapi", "pixsum_red", "pixsum_green", "pixsum_dapi", "range_red", "range_green", "range_dapi", "std_red", "std_green", "std_dapi", "sum_red", "sum_green", "sum_dapi", "sum2_red", "sum2_green", "sum2_dapi"]

In [18]:
res[values] = pd.concat([res[i].astype(str).str.replace('  ','0').replace(' ','0').replace('', '0').replace('None',"0").astype(float) for i in values], axis=1)

In [19]:
res["new_id"]= res['filename'] + "--" + res['image'].str.replace('.czi','').str.replace('Image ','').str.replace('Channel Alignment_P',"")+"--"+res['id'].astype(str)+"--"+res['parent_id'].astype(str)

<ipython-input-19-15ff780eabc4>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  res["new_id"]= res['filename'] + "--" + res['image'].str.replace('.czi','').str.replace('Image ','').str.replace('Channel Alignment_P',"")+"--"+res['id'].astype(str)+"--"+res['parent_id'].astype(str)


In [20]:
res["zsize"] = 1

In [21]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60945 entries, 0 to 60944
Data columns (total 36 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   image         60945 non-null  object 
 1   parent_id     60945 non-null  int64  
 2   id            60945 non-null  int64  
 3   area          60945 non-null  float64
 4   x             60945 non-null  float64
 5   y             60945 non-null  float64
 6   z             60945 non-null  float64
 7   count_red     60945 non-null  float64
 8   count_green   60945 non-null  float64
 9   max_red       60945 non-null  float64
 10  max_green     60945 non-null  float64
 11  max_dapi      60945 non-null  float64
 12  mean_red      60945 non-null  float64
 13  mean_green    60945 non-null  float64
 14  mean_dapi     60945 non-null  float64
 15  min_red       60945 non-null  float64
 16  min_green     60945 non-null  float64
 17  min_dapi      60945 non-null  float64
 18  pixsum_red    60945 non-nu

In [22]:
res = res.groupby('new_id').agg({
'area': "sum",
'x': "mean",
'y': "mean",
'z': "mean",
'count_red': "sum",
'count_green': "sum",
'max_red': "max",
'max_green': "max",
'max_dapi': "max",
'mean_red': "mean",
'mean_green': "mean",
'mean_dapi': "mean",
'min_red': "mean",
'min_green': "mean",
'min_dapi': "mean",
'pixsum_red': "sum",
'pixsum_green': "sum",
'pixsum_dapi': "sum",
'range_red': "max",
'range_green': "max",
'range_dapi': "max",
'std_red': "max",
'std_green': "max",
'std_dapi': "max",
'sum_red': "sum",
'sum_green': "sum",
'sum_dapi': "sum",
'sum2_red': "sum",
'sum2_green': "sum",
'sum2_dapi': "sum",
'zsize':"sum"
})

In [28]:
res['parent_id'] = [i.split('--')[-1] for i in res.index]
res['exp'] = [i.split('--')[0].split('-w')[0] for i in res.index]
res['treat'] = [i.split('--')[0].split('-w')[1] for i in res.index]
res['group'] = ['--'.join(i.split('--')[:-2]) for i in res.index]
res['id'] = ['--'.join(i.split('--')[-2]) for i in res.index]
res = res.reset_index(drop=True)

In [31]:
res['id'] = [i.split('--')[-1] for i in res.id]

In [29]:
set(res.exp)

{'VHL_MED1-MYC', 'VHL_MEF2C-MEF2D', 'VHL_MEF2C-MYC', 'VHL_MEF2D-MYC'}

In [ ]:
res

In [32]:
res

,area,x,y,z,count_red,count_green,max_red,max_green,max_dapi,mean_red,...,sum_dapi,sum2_red,sum2_green,sum2_dapi,zsize,parent_id,exp,treat,group,id
0,1.383583e+06,212.189340,224.153651,0.000,0.0,0.0,3325.0,5426.0,4612.0,1040.844364,...,1362378.0,2.165616e+09,8.793782e+09,2.543456e+09,43,4,VHL_MED1-MYC,DMSO,VHL_MED1-MYC-wDMSO--13_SIM_1-1,10
1,5.252128e+05,204.702775,196.176448,0.000,0.0,0.0,2072.0,5786.0,5009.0,612.702122,...,616678.0,4.278010e+08,3.218956e+09,1.467239e+09,15,5,VHL_MED1-MYC,DMSO,VHL_MED1-MYC-wDMSO--13_SIM_1-1,10
2,1.526645e+06,199.288170,229.900728,0.000,0.0,0.0,3292.0,8425.0,4414.0,1065.786607,...,1281964.0,2.526792e+09,1.471812e+10,2.341142e+09,41,4,VHL_MED1-MYC,DMSO,VHL_MED1-MYC-wDMSO--13_SIM_1-1,11
3,5.683273e+05,186.506854,207.208676,0.000,0.0,0.0,2719.0,7084.0,3083.0,893.834543,...,289827.0,7.182536e+08,5.197207e+09,3.774690e+08,15,5,VHL_MED1-MYC,DMSO,VHL_MED1-MYC-wDMSO--13_SIM_1-1,11
4,1.519786e+06,217.573480,235.780799,0.000,0.0,0.0,2977.0,6256.0,4191.0,1153.122827,...,1452794.0,2.501556e+09,1.212792e+10,2.599391e+09,40,4,VHL_MED1-MYC,DMSO,VHL_MED1-MYC-wDMSO--13_SIM_1-1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9038,5.723418e+09,216.915475,417.728055,56.000,8.0,20.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,91,1,VHL_MEF2D-MYC,VHL,VHL_MEF2D-MYC-wVHL--5_SIM_2-2,3
9039,3.106085e+08,202.136183,429.552692,45.400,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,5,1,VHL_MEF2D-MYC,VHL,VHL_MEF2D-MYC-wVHL--5_SIM_2-2,4
9040,4.918971e+05,191.271395,470.314529,49.375,0.0,0.0,6373.0,165.0,11591.0,3457.348770,...,2443570.0,6.953933e+09,5.027950e+05,1.742147e+10,8,5,VHL_MEF2D-MYC,VHL,VHL_MEF2D-MYC-wVHL--5_SIM_2-2,6
9041,5.095348e+04,521.500000,213.384615,0.000,0.0,0.0,299.0,1697.0,7621.0,126.923077,...,212780.0,1.434248e+06,8.325796e+07,1.075235e+09,2,6,VHL_MEF2D-MYC,VHL,VHL_MEF2D-MYC-wVHL--5_SIM_2-2,7


In [44]:
#removing known noise from samples with a nucleus in only one zstack
ind = []
for i in res[(res["parent_id"]=='1') & (res.zsize<8)][['group','id']].values:
    ind += res[(res.group==i[0])&(res.parent_id==i[1])].index.tolist()
res = res.drop(index=ind).reset_index(drop=True)
res = res.drop(index=res[(res["parent_id"]=='1') & (res.zsize < 8)].index).reset_index(drop=True)

In [53]:
for val in set(res.exp): 
    print('______________________')
    print(val)
    print("nucleus in DMSO/VHL")
    dmso = len(res[(res['exp']==val) & (res['parent_id']=='1') & (res['treat']=="DMSO")])
    vhl = len(res[(res['exp']==val) & (res['parent_id']=='1') & (res['treat']=="VHL")])
    print(dmso, vhl)
    print("foldchange in VHL:")
    print(' red        green')
    print(((res[(res['exp']==val) & (res['parent_id']=='1') & (res['treat']=="DMSO")][['count_red','count_green']].mean()/dmso)/(res[(res['exp']==val) & (res['parent_id']=='1') & (res['treat']=="VHL")][['count_red','count_green']].mean()/vhl)).values)

______________________
VHL_MEF2C-MEF2D
nucleus in DMSO/VHL
16 10
foldchange in VHL:
 red        green
[0.23103784 0.45327844]
______________________
VHL_MEF2C-MYC
nucleus in DMSO/VHL
18 9
foldchange in VHL:
 red        green
[0.35294118 1.42824074]
______________________
VHL_MED1-MYC
nucleus in DMSO/VHL
12 14
foldchange in VHL:
 red        green
[0.7346583  2.58687364]
______________________
VHL_MEF2D-MYC
nucleus in DMSO/VHL
11 18
foldchange in VHL:
 red        green
[2.89800188 1.78624528]


In [1]:
res[(res['exp']==val) & (res['parent_id']!='1') & (res['treat']=="DMSO")].zsiz

NameError: name 'res' is not defined